In [13]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg

In [32]:
crimes = pd.read_csv("/Users/Khanna/Downloads/crime_cleaned_parth.csv")

In [40]:
calls = np.load("/Users/Khanna/Downloads/cleaned_data.npy")
calls = pd.DataFrame(calls)

In [44]:
calls.columns = ['Type', 'Category', 'DateTime', 'Approach','Borough']

In [34]:
def timeOfDay(time):
  if(time==None):
    return "other"
  time = int(time)
  if(time>=21 or time <=4):
    return "nuit"
  if(time>=5 and time <=12):
    return "jour"
  if(time>=13 and time <=20):
    return "soir"

In [47]:
calls['Date'], calls['Time'] = calls['DateTime'].str.split('T', 1).str
del calls['DateTime']
calls['Time'] = (calls['Time'].str[:2])

,Type,Category,Approach,Borough,Date,Time
0,Information,Fermeture d'entrée d'eau,Téléphone,SERVICE DE LA CONCERTATION DES ARRONDISSEMENTS,2016-01-01,02
1,Information,Fuite d'eau,Personne,SERVICE DE LA CONCERTATION DES ARRONDISSEMENTS,2016-01-01,05
2,Information,Éclairage existant - Entretien,Téléphone,ST-LAURENT,2016-01-01,06
3,Information,Fuite d'eau,Téléphone,SERVICE DE LA CONCERTATION DES ARRONDISSEMENTS,2016-01-01,06
4,Information,Éclairage existant - Entretien,Téléphone,ST-LAURENT,2016-01-01,07
5,Information,Feux de circulation - Entretien,Téléphone,ST-LAURENT,2016-01-01,07
6,Information,Panneau de signalisation entretien prioritaire,Téléphone,ST-LAURENT,2016-01-01,07
7,Information,Fuite d'eau,Téléphone,SERVICE DE L EXPÉRIENCE CITOYENNE ET DES COMMU...,2016-01-01,07
8,Information,Collecte de déchets,Téléphone,LASALLE,2016-01-01,09
9,Information,Collecte de déchets,Téléphone,LASALLE,2016-01-01,09


In [49]:
calls['Time'] = calls['Time'].apply(timeOfDay)

ValueError: invalid literal for int() with base 10: 'nuit'

In [73]:
calls.groupby('Category').count()

,Type,Approach,Borough,Date,Time
Category,,,,,
* Fuite d'eau (arr.),9,9,9,9,9
*01 - Demande d'intervention 5700,654,654,654,654,654
*03 - Affichage illégal,1,1,1,1,1
*08 - Porter assistance au SSIM,2,2,2,2,2
*09 - Porter assistance aux services municipaux,2,2,2,2,2
*11 - Bruit,3,3,3,3,3
*12 - Danger ou débris sur la voie publique,5,5,5,5,5
*13 - Demande de circuler,4,4,4,4,4
"*15 - Expulsion chalets, parcs, stationnements",1,1,1,1,1


In [71]:
calls.groupby('Type').count()

,Category,Approach,Borough,Date,Time
Type,,,,,
Commentaire,10082,10082,10082,10082,10082
Information,1420351,1420351,1420351,1420351,1420351
Plainte,18395,18395,18395,18395,18395
Requete,1342865,1340431,1342865,1342865,1342865


In [55]:
crimes[:10]

,Unnamed: 0,CATEGORIE,DATE,QUART,PDQ,LONGITUDE,LATITUDE
0,0,Vol de véhicule à moteur,2018-09-13,jour,30.0,-73.626778,45.567780
1,1,Vol de véhicule à moteur,2018-04-30,jour,30.0,-73.626778,45.567780
2,2,Vol de véhicule à moteur,2018-09-01,nuit,7.0,-73.685928,45.519122
3,3,Méfait,2017-07-21,jour,21.0,1.000000,1.000000
4,4,Méfait,2017-07-29,jour,12.0,1.000000,1.000000
5,5,Vol de véhicule à moteur,2017-07-30,nuit,21.0,1.000000,1.000000
6,6,Méfait,2017-07-30,jour,38.0,-73.591457,45.516776
7,7,Vols qualifiés,2017-07-30,jour,21.0,1.000000,1.000000
8,8,Vol dans / sur véhicule à moteur,2017-08-01,jour,39.0,-73.635117,45.602873
9,9,Méfait,2017-08-01,jour,21.0,1.000000,1.000000


In [23]:
coordinates = (45.567780, -73.626778)
results = rg.search(coordinates)
print(results)

[OrderedDict([('lat', '45.58773'), ('lon', '-73.59501'), ('name', 'Saint-Leonard'), ('admin1', 'Quebec'), ('admin2', 'Montreal'), ('cc', 'CA')])]


KeyError: 'Longitude'